## Period 1

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pandas_profiling as pp

pd.options.display.max_info_columns = 999
pd.options.display.max_info_rows = 999
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
pd.options.display.max_seq_items = 999
pd.options.display.float_format = '{:.3f}'.format

import warnings
warnings.filterwarnings("ignore")

### tfct

In [2]:
df_tfct = pd.read_csv(r'C:\Users\User\Documents\Kiyvstar\Вig Data School 4.0\projects\data\tfct_period_1.csv')

In [3]:
df_tfct['time'] = df_tfct['time'].apply(lambda x: x.split('.')[0])

In [4]:
df_tfct.head()

,id,product_class_key,direction_key,duration_nval,success_ind,operator,b_id,period,day,time
0,1289,21,1,0.000,0,Vodafone,7110372,1,9,09:18:03
1,1289,21,2,0.000,0,Vodafone,7110372,1,26,12:14:25
2,1289,21,1,15.000,1,Vodafone,7110372,1,16,09:31:11
3,1289,21,1,0.000,0,Vodafone,7110372,1,8,09:32:22
4,1289,21,1,0.000,0,Vodafone,7110372,1,5,19:16:53


### Aggregations (period to day)

In [5]:
%%time
# count product_class_key  and uniqie product_class_key in day
product_class_key =  pd.DataFrame(df_tfct.groupby(['id', 'day'])['product_class_key'].agg({
    #'product_class_key_count': lambda x: len(x),
    'product_class_key_unique': lambda x: x.nunique()})
                                 )

Wall time: 6min 42s


In [6]:
%%time
# count direction_key  and uniqie direction_key in day
direction_key = pd.DataFrame(df_tfct.groupby(['id', 'day'])['direction_key'].agg({#'direction_key_count': lambda x: len(x), 
                                                                       'direction_key_unique':  lambda x: x.nunique()}
                                                                       ))

Wall time: 6min 6s


In [7]:
%%time 

duration_nval = pd.DataFrame(df_tfct.groupby(['id', 'day'])['duration_nval'].agg(
    {#'duration_nval_max': lambda x: max(x), 
     #'duration_nval_count': lambda x: len(x), # count duration in day
     'duration_nval_mean': lambda x: np.mean(x), # mean duration in day
     'duration_nval_std': lambda x: np.std(x) # std duration in day
    } ) )


Wall time: 5min 27s


In [8]:
# %%time 
# count 1
# success_ind = pd.DataFrame(df_tfct.groupby(['id', 'day'])['success_ind'].agg({'success_ind_sum': lambda x: sum(x)} ))

In [9]:
operator = pd.DataFrame(df_tfct.groupby(['id', 'day'])['operator'].nunique())  # count unique operator for id

In [10]:
%%time
# b_id direction_key  and uniqie direction_key in day
b_id = pd.DataFrame(df_tfct.groupby(['id', 'day'])['b_id'].agg({#'b_id_count': lambda x: len(x), 
                                                                       'b_id_unique':  lambda x: x.nunique()}
                                                                       ))

Wall time: 6min 21s


In [11]:
count_call_to_day = df_tfct.groupby(['id', 'day'])['time'].count() #count call in day

### worktime

In [12]:
df_1 = df_tfct[(df_tfct.time >= '08:45:00') & (df_tfct.time <= '18:15:00')]

In [13]:
count_call_to_worktime = pd.DataFrame(df_1.groupby(['id', 'day'])['time'].count()) #count call in worktime

In [14]:
%%time 

duration_nval_worktime = pd.DataFrame(df_1.groupby(['id', 'day'])['duration_nval'].agg(
    {#'duration_nval_max_worktime': lambda x: max(x), 
     #'duration_nval_count_worktime': lambda x: len(x), # count duration in day
     'duration_nval_mean_worktime': lambda x: np.mean(x), # mean duration in day
     'duration_nval_std_worktime': lambda x: np.std(x) # std duration in day
    } ) )

Wall time: 5min 20s


In [15]:
del df_1, df_tfct

In [16]:
df_2 =product_class_key.merge(direction_key, on=['id', 'day']).merge(duration_nval, on=['id', 'day']) \
.merge(b_id, on=['id', 'day']).merge(count_call_to_day, on=['id', 'day']) \
.merge(count_call_to_worktime, on=['id', 'day'], suffixes=(' ', '_worktime')) \
.merge(duration_nval_worktime, on=['id', 'day'])

#.merge(success_ind, on=['id', 'day']).merge(operator, on=['id', 'day']) \

In [17]:
df_2.head()

product_class_key_unique  direction_key_unique  duration_nval_mean  \
id day                                                                       
1  1                           2                     1              90.833   
   3                           1                     3             160.833   
   4                           1                     1             211.000   
   5                           1                     3              60.273   
   6                           1                     1             260.000   

        duration_nval_std  b_id_unique  time   time_worktime  \
id day                                                         
1  1              191.592            5      6              5   
   3              448.999            7     12             11   
   4              187.000            2      2              1   
   5              121.335            4     11              6   
   6              358.372            2      5              5   

        duration_nval_mean_worktime  duration_nval_std_worktime  
id day                                                           
1  1                          5.200                       7.111  
   3                        170.636                     467.733  
   4                         24.000                       0.000  
   5                         86.333                     158.057  
   6                        260.000                     358.372

In [18]:
df_2.reset_index(inplace=True)

In [19]:
df_2.shape

(3123563, 11)

In [20]:
%%time

rejected = pp.ProfileReport(df_2).get_rejected_variables()

Wall time: 1min 31s


In [21]:
%%time

pp.ProfileReport(df_2)

Wall time: 1min 24s


Number of variables,11
Number of observations,3123563
Total Missing (%),0.0%
Total size in memory,262.1 MiB
Average record size in memory,88.0 B
Numeric,10
Categorical,0
Boolean,0
Date,0
Text (Unique),0
Rejected,1


In [22]:
rejected

['time_worktime']

In [23]:
df_2.shape

(3123563, 11)

In [24]:
df_2.columns

Index(['id', 'day', 'product_class_key_unique', 'direction_key_unique',
       'duration_nval_mean', 'duration_nval_std', 'b_id_unique', 'time ',
       'time_worktime', 'duration_nval_mean_worktime',
       'duration_nval_std_worktime'],
      dtype='object')

In [25]:
df_2.rename(columns={'time ': 'time'}, inplace=True)

In [26]:
#df_2['duration_nval_mean'] =df_2['duration_nval_mean'] /60 #minutes to hour

### Aggregations (day to id)

In [27]:
df_2.head()

,id,day,product_class_key_unique,direction_key_unique,duration_nval_mean,duration_nval_std,b_id_unique,time,time_worktime,duration_nval_mean_worktime,duration_nval_std_worktime
0,1,1,2,1,90.833,191.592,5,6,5,5.200,7.111
1,1,3,1,3,160.833,448.999,7,12,11,170.636,467.733
2,1,4,1,1,211.000,187.000,2,2,1,24.000,0.000
3,1,5,1,3,60.273,121.335,4,11,6,86.333,158.057
4,1,6,1,1,260.000,358.372,2,5,5,260.000,358.372


In [28]:
%%time 

count_day_to_period = pd.DataFrame(df_2.groupby(['id'])['day'].count()) #count call day in period


call_to_period =  pd.DataFrame(df_2.groupby(['id'])['time'].agg({'mean_call_to_period': lambda x: np.mean(x),
                                                                      'std_call_to_period': lambda x: np.std(x),
                                                               # 'count_call_to_period': lambda x: len(x)
                                                                }))
    
call_to_period_work =  pd.DataFrame(df_2.groupby(['id'])['time_worktime'].agg(
    {'mean_call_to_period_worktime': lambda x: np.mean(x),
    'std_call_to_period_worktime': lambda x: np.std(x), 
    # 'count_call_to_period_worktime': lambda x: len(x)
    }))

duration_nval_mean_to_period = pd.DataFrame(df_2.groupby(['id'])['duration_nval_mean'].mean()) # mean  duration_nval in period
duration_nval_std_to_period = pd.DataFrame(df_2.groupby(['id'])['duration_nval_std'].std()) # mean  duration_nval in period

Wall time: 25.2 s


In [29]:
df_tfct_preparation = count_day_to_period.merge(call_to_period, on='id').merge(call_to_period_work, on='id') \
.merge(duration_nval_mean_to_period, on='id').merge(duration_nval_std_to_period, on='id') 

In [30]:
df_tfct_preparation.head()

,day,mean_call_to_period,std_call_to_period,mean_call_to_period_worktime,std_call_to_period_worktime,duration_nval_mean,duration_nval_std
id,,,,,,,
1,29,9.793,6.419,7.138,6.050,108.995,116.144
2,30,28.500,18.222,20.333,13.753,90.647,128.357
3,30,21.667,10.193,13.967,6.258,62.708,77.978
4,30,14.633,6.877,9.300,4.954,136.506,206.420
5,28,14.500,9.348,12.036,7.980,47.488,40.557


In [31]:
df_tfct_preparation['ratio_call'] = df_tfct_preparation['mean_call_to_period_worktime'] / df_tfct_preparation['mean_call_to_period']  # relation call in worktime to day

In [32]:
df_tfct_preparation.reset_index(inplace=True)

In [33]:
df_tfct_preparation.shape

(128786, 9)

In [34]:
df_tfct_preparation.drop_duplicates(inplace=True)

In [35]:
df_tfct_preparation.shape

(128786, 9)

### df_base_station

In [36]:
df_base_station = pd.read_csv(r'C:\Users\User\Documents\Kiyvstar\Вig Data School 4.0\projects\data\base_station.csv')

In [37]:
df_base_station.shape

(758101, 6)

In [38]:
df_base_station.columns

Index(['id', 'period', 'base_station_cnt', 'city_cnt', 'area_cnt',
       'region_cnt'],
      dtype='object')

In [39]:
set(df_base_station.period)

{1, 2, 3, 4, 5, 6}

In [40]:
df_base_station = df_base_station[df_base_station.period == 1] # period 1 base_station

In [41]:
df_base_station['region_city'] = df_base_station['region_cnt'] /df_base_station['area_cnt']
df_base_station['area_city'] = df_base_station['city_cnt'] /df_base_station['area_cnt']

In [42]:
pp.ProfileReport(df_base_station)

Number of variables,9
Number of observations,130814
Total Missing (%),0.0%
Total size in memory,9.0 MiB
Average record size in memory,72.0 B
Numeric,5
Categorical,0
Boolean,0
Date,0
Text (Unique),0
Rejected,4


In [43]:
df_base_station.drop(['region_cnt', 'city_cnt'], axis=1, inplace=True)

In [44]:
df_base_station.head()

,id,period,base_station_cnt,area_cnt,region_city,area_city
0,1,1,143.000,10.000,0.600,2.200
6,2,1,151.000,3.000,0.333,5.667
12,3,1,141.000,2.000,0.500,1.500
18,4,1,315.000,16.000,0.250,3.688
24,5,1,182.000,2.000,0.500,1.000


### alpha_num

In [45]:
df_alpha_num = pd.read_csv(r'C:\Users\User\Documents\Kiyvstar\Вig Data School 4.0\projects\data\alpha_num.csv')

In [46]:
df_alpha_num.shape

(734947, 4)

In [47]:
set(df_alpha_num.period)

{1, 2, 3, 4, 5, 6}

In [48]:
df_alpha_num = df_alpha_num[df_alpha_num.period == 1]

In [49]:
df_alpha_num['distinct_number/sms'] = df_alpha_num['distinct_number_cnt'] / df_alpha_num['sms_cnt']

In [50]:
df_alpha_num.head()

,id,period,sms_cnt,distinct_number_cnt,distinct_number/sms
0,1,1,86,17,0.198
6,2,1,48,5,0.104
12,3,1,76,9,0.118
18,4,1,110,4,0.036
24,5,1,58,2,0.034


In [51]:
pp.ProfileReport(df_alpha_num)

Number of variables,6
Number of observations,128772
Total Missing (%),0.0%
Total size in memory,5.9 MiB
Average record size in memory,48.0 B
Numeric,4
Categorical,0
Boolean,0
Date,0
Text (Unique),0
Rejected,2


### dm

In [52]:
df_dm = pd.read_csv(r'C:\Users\User\Documents\Kiyvstar\Вig Data School 4.0\projects\data\dm.csv')

In [53]:
set(df_dm.period)

{1, 2, 3, 4, 5, 6, 7}

In [54]:
df_dm = df_dm[df_dm.period == 1]

In [55]:
# %%time

# pp.ProfileReport(df_dm)

In [56]:
%%time

rejected = pp.ProfileReport(df_dm).get_rejected_variables()

Wall time: 1min 58s


In [57]:
df_dm.drop(rejected, axis=1, inplace=True)

In [58]:
df_dm.shape

(134433, 76)

In [59]:
df_dm.head()

,id,period,preffered_location_region,refilled_amount,activity_voice_days_ind,activity_data_days_ind,activity_sms_days_ind,voice_on_net_ind,voice_off_net_ind,international_calls_ind,sms_ind,internet_ind,act_off_net_voice_days_ind,on_net_outgoing_calls_min_we,on_net_outgoing_calls_min_bd,off_net_outgoing_calls_min_we,off_net_outgoing_calls_min_bd,on_net_outgoing_calls_count_we,on_net_outgoing_calls_count_bd,off_net_out_calls_count_we,off_net_out_calls_count_bd,on_net_out_revenue_core_we,on_net_out_revenue_core_bd,off_net_out_revenue_core_we,off_net_out_revenue_core_bd,outgoing_sms_count,outgoing_sms_count_we,sms_revenue_core_we,sms_revenue_core_bd,roaming_sms_revenue_core_we,roaming_sms_revenue_core_bd,outgoing_mms_count,outgoing_mms_count_we,outgoing_mms_count_bd,int_outgoing_calls_min,roaming_outgoing_calls_min,other_outgoing_calls_min,outgoing_sms_off_net_count,outgoing_sms_int_count,roaming_outgoing_sms_count,mobile_data_mb,roaming_mobile_data_mb,total_incoming_calls_min,off_net_incoming_calls_min,int_incoming_calls_min,roaming_incoming_calls_min,int_outgoing_calls_count,roaming_outgoing_calls_count,total_incoming_calls_count,off_net_incoming_calls_count,int_incoming_calls_count,roaming_incoming_calls_count,total_outgoing_revenue_core,periodic_fees_core,non_periodic_fees_core,int_outgoing_revenue_core,roaming_outgoing_revenue_core,roaming_incoming_revenue_core,mobile_data_revenue_core,other_vas_revenue_core,total_itc_revenue,off_net_itc_revenue,total_itc_cost,off_net_itc_cost,total_sms_revenue,off_net_sms_revenue,total_sms_cost,off_net_sms_cost,international_sms_cost,sms_revenue_on_net_core,sms_revenue_off_net_core,mobile_data_3g_mb,mobile_data_200_free_mb,roaming_ind,activity_data_4g_days_cnt,mobile_data_4g_mb
0,100349,1,8.000,0.000,2.000,11.000,1.000,1.000,1.000,0.000,1.000,1.000,2.000,0.000,0.100,2.750,4.050,0.000,1.000,3.000,3.000,0.000,0.000,0.000,0.000,1.000,1.000,2.200,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,1810.994,798.789,6.230,1.080,0.000,1.780,0.000,0.000,10.000,3.000,0.000,3.000,612.200,100.000,0.000,0.000,0.000,90.000,0.000,0.000,9.586,9.586,0.816,0.816,2.066,0.050,1.480,0.000,1.480,0.000,0.000,1764.994,1330.042,1.000,0.000,0.000
6,100402,1,4.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
12,100576,1,8.000,0.000,20.000,0.000,2.000,1.000,1.000,0.000,1.000,0.000,10.000,0.000,163.770,0.000,16.770,0.000,199.000,0.000,16.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.090,1.000,0.000,0.000,0.000,0.000,57.930,1.850,0.000,0.000,0.000,0.000,58.000,2.000,0.000,0.000,150.000,150.000,0.000,0.000,0.000,0.000,0.000,0.000,0.222,0.222,2.020,2.020,2.107,0.150,0.050,0.050,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
18,100580,1,8.000,0.000,29.000,0.000,0.000,1.000,1.000,0.000,0.000,0.000,10.000,48.830,868.120,7.980,34.380,14.000,664.000,1.000,13.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.150,0.000,0.000,0.000,0.000,0.000,207.830,5.200,0.000,0.000,0.000,0.000,134.000,4.000,0.000,0.000,150.000,150.000,0.000,0.000,0.000,0.000,0.000,0.000,0.660,0.660,5.088,5.088,0.200,0.200,0.100,0.100,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
24,100684,1,10.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2.870,1.280,0.000,0.000,0.000,0.000,3.000,1.000,0.000,0.000,5.310,0.000,5.310,0.000,0.000,0.000,0.000,0.000,0.154,0.154,0.000,0.000,

### decision_maker

In [60]:
df_decision_maker = pd.read_csv(r'C:\Users\User\Documents\Kiyvstar\Вig Data School 4.0\projects\samples\Decision_maker.csv',
                                sep=';')

In [61]:
# df_decision_maker = df_decision_maker.merge(df_base_station, on=['id'], how='left').merge(df_handset_type, on='id', how='left') \
#                             .merge(df_alpha_num, on=['id', 'period'], how='left').merge(df_dm, on=['id', 'period'], how='left')

In [62]:
df_decision_maker.head()

,id,target
0,70000,0
1,80000,0
2,90000,0
3,100000,0
4,110000,0


In [63]:
df_decision_maker.shape

(45200, 2)

In [64]:
df_clear = df_decision_maker.merge(df_base_station, on=['id'], how='left') \
                            .merge(df_alpha_num, on=['id', 'period'], how='left').merge(df_tfct_preparation, on='id') \
                            .merge(df_dm, on=['id', 'period'], how='left')

In [65]:
df_clear.fillna(df_clear.median(), inplace = True)

In [66]:
rejected = pp.ProfileReport(df_clear).get_rejected_variables()

In [67]:
rejected

['mean_call_to_period_worktime',
 'std_call_to_period_worktime',
 'outgoing_sms_off_net_count',
 'roaming_outgoing_calls_count',
 'roaming_incoming_calls_count',
 'sms_revenue_on_net_core',
 'sms_revenue_off_net_core']

In [68]:
df_clear.drop(rejected, axis=1, inplace=True)

In [69]:
df_clear.shape

(38746, 85)

In [70]:
df_clear.to_csv(r'C:\Users\User\Documents\Kiyvstar\Вig Data School 4.0\projects\data\data_clear_1.csv', index=False)